In [1]:
import covalent as ct
import pickle
import os
import pennylane.numpy as np
import torch
from itertools import cycle
import matplotlib.pyplot as plt
import pennylane as qml
from itertools import combinations
from pytorch_scipy_optimizer import Optimizer

# Need this to resolve paths pater
base_path = os.getcwd()

In [2]:
@ct.electron
def load_data_from_pickle(path):
    data = pickle.load(open(path, 'rb'))
    return data

In [3]:
Xtr = load_data_from_pickle("".join([base_path, "/data/Xtr.pickle"])) # 3d array

In [4]:
@ct.electron
def data_santity_check(data):
    # Only accept 3d data. (num_series, num_features, num_time_points)
    X = np.asarray(data)
    # Assume 3d array is list of nd-feature time series
    if len(X.shape) != 3:
        raise TypeError("Dimensions of X must be 3 dimensional: (num_series, num_features, num_time_points)")

In [ ]:
data_santity_check(Xtr)

In [5]:
class DataGetter:
    def __init__(self, X, batch_size, n_batch_iterations=1000):
        self.data = []
        self._init_data(
            cycle(torch.utils.data.DataLoader(X, batch_size=batch_size, shuffle=True)),
            n_batch_iterations)

    def _init_data(self, iterator, n_batch_iterations=1000):
        x = next(iterator, None)
        for i in range(n_batch_iterations):
            self.data.append(x)
            x = next(iterator, None)

    def __next__(self):
        return self.data.pop()

In [6]:
@ct.electron
def get_series_training_dataloader(Xtr, n_series_batch, batch_iterations=1000):
    x_cycler = DataGetter(Xtr, n_series_batch, batch_iterations)
    return x_cycler

@ct.electron
def get_timepoint_training_dataloader(Xtr, n_t_batch, batch_iterations=1000):
    n_time_points = Xtr.shape[2]
    T  = torch.tensor(np.arange(n_time_points))
    t_cycler = DataGetter(T, n_t_batch, batch_iterations)
    return t_cycler

In [ ]:
x_cycler = get_series_training_dataloader(Xtr, 10)
t_cycler = get_timepoint_training_dataloader(Xtr, 10)

In [7]:
@ct.electron
def arctan_penalty(sigma, contraction_hyperparameter):
    prefac = 1/(np.pi)
    sum_terms = torch.arctan(2*np.pi*contraction_hyperparameter*torch.abs(sigma))
    mean = sum_terms.mean()
    return prefac*mean

In [ ]:
sigma = torch.tensor([100, 100, 100])
pen = arctan_penalty(sigma, 1)

In [8]:
@ct.electron
def create_diagonal_circuit(D, n, k=None):
    if k is None:
        k = n
    cnt = 0
    for i in range(1, k + 1):
        for comb in combinations(range(n), i):
            if len(comb) == 1:
                qml.RZ(D[cnt], wires=[comb[0]])
                cnt += 1
            elif len(comb) > 1:
                cnots = [comb[i : i + 2] for i in range(len(comb) - 1)]
                for j in cnots:
                    qml.CNOT(wires=j)
                qml.RZ(D[cnt], wires=[comb[-1]])
                cnt += 1
                for j in cnots[::-1]:
                    qml.CNOT(wires=j)

In [ ]:
D = torch.tensor([2, 2, 2])
n_qubits = 2
create_diagonal_circuit(D, n_qubits, 2)

In [ ]:
@ct.electron
def get_device(n_qubits):
    device = qml.device('default.qubit', wires=n_qubits, shots=None)
    return device

In [ ]:
device = get_device(n_qubits)

In [9]:
@ct.electron
@qml.qnode(device, interface='torch')
def get_anomaly_expec(x, t, D, alpha, wires, k, embed_func, transform_func, diag_func, observable,
                      embed_func_params={}, transform_func_params={}):
    embed_func(x, wires=wires, **embed_func_params)
    transform_func(alpha, wires, **transform_func_params)
    diag_func(D * t, n_qubits, k=k)
    qml.adjoint(transform_func)(alpha, wires=range(n_qubits), **transform_func_params
        )
    coeffs = np.ones(len(wires))/len(wires)
    H = qml.Hamiltonian(coeffs, observable)
    return qml.expval(H)

NameError: name 'device' is not defined

In [ ]:
observable = [qml.PauliZ(i) for i in range(n_qubits)]
x = torch.tensor([0.1, 0.7])
t = torch.tensor([0.1])
D = torch.tensor([0.1, 0.1, 0.1])
k = 2
transform_func = qml.templates.StronglyEntanglingLayers
alpha_weights_shape = transform_func.shape(3, 2)
alpha = torch.tensor(np.random.uniform(0, 2 * np.pi, size=alpha_weights_shape), requires_grad=True
            ).type(torch.DoubleTensor)
embed_func = qml.templates.AngleEmbedding

get_anomaly_expec(x, t, D, alpha, range(n_qubits), k, embed_func, transform_func, create_diagonal_circuit, observable)

In [10]:
@ct.electron
def sample_D(sigma, mu):
    D = torch.normal(mu, sigma.abs())
    return D

In [ ]:
D = sample_D(torch.tensor([1.0, 2.0, 3.0]), torch.tensor([0.0, 3.1, 4.2]))

In [11]:
@ct.electron
def get_single_point_cost(x, t, alpha, q, D_sample_func, sigma, mu, N_E, wires, k, embed_func, transform_func, diag_func, observable,
                      embed_func_params={}, transform_func_params={}):
    expecs = torch.zeros(N_E)
    for i in range(N_E):
        D = D_sample_func(sigma, mu)
        expec = get_anomaly_expec(x, t, D, alpha, wires, k, embed_func, transform_func, diag_func, observable,
                                   embed_func_params={}, transform_func_params={})
        expecs[i] = expec
    mean = expecs.mean()
    single_point_cost = (q - mean)**2/4
    return single_point_cost

In [ ]:
q = torch.tensor([-1])
mu = torch.tensor([0.1, 0.2, 0.1])
sigma = torch.tensor([0.6, 4.1, 2.0])
N_E = 10
wires = range(n_qubits)
diag_func = create_diagonal_circuit
get_single_point_cost(x, t, alpha, q, sample_D, sigma, mu, N_E, wires, k, embed_func, transform_func, diag_func, observable,)

In [12]:
@ct.electron
def get_time_series_cost(xt, alpha, q, D_sample_func, sigma, mu, N_E, wires, k, embed_func,
                         transform_func, diag_func, observable, t_cycler, embed_func_params={},
                         transform_func_params={}):
    if t_cycler is None:
        t_idxs = np.arange(xt.shape[1])
        ts = np.linspace(0.1, 2 * np.pi, xt.shape[1], endpoint=True)
    else:
        t_idxs = next(t_cycler)
        ts = np.linspace(0.1, 2 * np.pi, xt.shape[1], endpoint=True)[t_idxs]
    xt_batch = xt[:, t_idxs]
    xfunct = zip(xt_batch.T, ts)
    a_func_t = \
        [get_single_point_cost(x, t, alpha, q, sample_D, sigma, mu, N_E, wires,
                               k, embed_func, transform_func, diag_func, observable) for x, t in xfunct]
    single_time_series_cost = torch.tensor(a_func_t, requires_grad=True).mean()
    return single_time_series_cost

In [ ]:
xt = Xtr[0]
t_cycler = get_timepoint_training_dataloader(Xtr, 10)
single_time_series_cost = \
     get_time_series_cost(xt, alpha, q, sample_D, sigma, mu, N_E, wires, k, embed_func,
                         transform_func, diag_func, observable, t_cycler, embed_func_params={},
                         transform_func_params={})
print(single_time_series_cost)

In [13]:
@ct.electron
def get_loss(alpha, q, D_sample_func, sigma, mu, N_E, wires, k, embed_func,
            transform_func, diag_func, observable, t_cycler, x_cycler, penalty, tau, embed_func_params={},
            transform_func_params={}):

    X_batch = next(x_cycler)
    single_costs = torch.zeros(X_batch.shape[0])
    for i, xt in enumerate(X_batch):
        single_time_series_cost = \
            get_time_series_cost(xt, alpha, q, sample_D, sigma, mu, N_E, wires, k, embed_func,
                                 transform_func, diag_func, observable, t_cycler, embed_func_params={},
                                 transform_func_params={})
        single_costs[i] = single_time_series_cost
    loss = 0.5*single_costs.mean() + penalty(sigma, tau)
    return loss

In [ ]:
penalty = arctan_penalty
tau = 1
loss = get_loss(alpha, q, sample_D, sigma, mu, N_E, wires, k, embed_func,
            transform_func, diag_func, observable, t_cycler, x_cycler, penalty, tau)
print(loss)

In [14]:
@ct.electron
def get_initial_parameters(transform_func_shape, n_qubits):
    initial_alpha =\
         torch.tensor(np.random.uniform(0, 2 * np.pi, size=transform_func_shape),
         requires_grad=True).type(torch.DoubleTensor)
    initial_mu = torch.tensor(
                np.random.uniform(0, 2 * np.pi, 2 ** (n_qubits) - 1)).type(torch.DoubleTensor)
    initial_sigma = torch.tensor(
                np.random.uniform(0, 2 * np.pi, 2 ** (n_qubits) - 1)).type(torch.DoubleTensor)

    initial_q = torch.tensor(np.random.uniform(-1, 1)).type(torch.DoubleTensor)
    init_parameters = {'alpha': initial_alpha, 'mu': initial_mu, 'sigma': initial_sigma, 'q': initial_q}
    return init_parameters

In [ ]:
n_qubits = 2
transform_func_layers = 3
transform_func_shape = qml.templates.StronglyEntanglingLayers.shape(transform_func_layers, n_qubits)
init_parameters = get_initial_parameters(transform_func_shape, n_qubits)
print(init_parameters)

In [15]:
@ct.electron
def train_model(alpha, q, D_sample_func, sigma, mu, N_E, wires, k, embed_func, n_qubits,
                transform_func, diag_func, observable, t_cycler, x_cycler, penalty, tau,
                optimizer_params, initial_parameters):
    f = lambda alpha, mu, sigma, q: get_loss(alpha=alpha, mu=mu, sigma=sigma, q=q,
                                             D_sample_func=D_sample_func, N_E=N_E, wires=range(n_qubits),
                                             k=k, embed_func=embed_func, transform_func=transform_func,
                                             diag_func=diag_func, observable=observable,
                                             t_cycler=t_cycler, x_cycler=x_cycler,
                                             penalty=penalty, tau=tau)
    optimizer = Optimizer(f, initial_parameters, optimizer_params)
    opt_params = optimizer.optimize()
    return {"opt_params": opt_params}

In [ ]:
initial_parameters = get_initial_parameters(transform_func_shape, n_qubits)
optimizer_params = {
                "method": "COBYLA",
                "options": {"disp": True, "maxiter": 1},
                "jac": False,
            }
opt_params =\
    train_model(alpha, q, sample_D, sigma, mu, N_E, wires, k, embed_func, n_qubits,
                transform_func, diag_func, observable, t_cycler, x_cycler, penalty, tau,
                optimizer_params, initial_parameters)
print(opt_params)        

In [ ]:
opt_params['opt_params']

In [16]:
@ct.electron
def get_anomaly_scores(Xte, normal_cost, penalty, tau, results_dict, D_sample_func, wires, k, embed_func,
                       transform_func, diag_func, observable, get_time_resolved=False):
    opt_params = results_dict['opt_params']
    alpha = opt_params['alpha']
    mu = opt_params['mu']
    sigma = opt_params['sigma']
    q = opt_params['q']
    scores = torch.zeros(Xte.shape[0])
    for i, yt in enumerate(Xte):
        single_time_series_cost =\
            get_time_series_cost(yt, alpha, q, D_sample_func, sigma, mu, N_E, wires, k, embed_func,
                         transform_func, diag_func, observable, t_cycler=None)
        scores[i] = (normal_cost - single_time_series_cost - penalty(sigma, tau)).abs()
        print('done', i)
    return scores

In [ ]:
Yte = load_data_from_pickle("".join([base_path, "/data/Xte_dirty_usdt_pos.pickle"]))
Yte = Yte[1:3, :, ::4]
normal_cost = 0
D_sample_func = sample_D
k = 2
n_qubits = 2
transform_func = qml.templates.StronglyEntanglingLayers
diag_func = create_diagonal_circuit
wires = range(n_qubits)
scores = get_anomaly_scores(Yte, normal_cost, penalty, tau, opt_params, D_sample_func, wires, k, embed_func,
                            transform_func, diag_func, observable, get_time_resolved=False)

In [ ]:
print(scores)

In [17]:
@ct.electron
def get_transform_func(type):
    if type == 'sel':
        transform_func = qml.StronglyEntanglingLayers
    return transform_func

@ct.electron
def get_embedding_func(type):
    if type == 'rx':
        embed_func = qml.templates.AngleEmbedding
    return embed_func

## Electrons all work locally, let's dispatch them at lattices...

In [19]:
@ct.lattice
def training_workflow(Xtr_path, n_series_batch, n_t_batch, transform_func_type, n_qubits, transform_func_layers,
                      embed_func_type='rx'):
    # load training data
    Xtr = load_data_from_pickle(Xtr_path)

    # check dimensions of input data are correct
    data_santity_check(Xtr)

    # Get dataloaders for series and time-point batches
    x_cycler = get_series_training_dataloader(Xtr, n_series_batch)
    t_cycler = get_timepoint_training_dataloader(Xtr, n_t_batch)

    # get penalty function
    pen = arctan_penalty

    # get transform_func
    transform_func = get_transform_func(transform_func_type)

    # get embedding func
    embed_func = get_embedding_func(embed_func_type)

    # get random initial parameters
    init_parameters = get_initial_parameters(transform_func.shape(transform_func_layers, n_qubits), n_qubits)    
    return pen

In [21]:
Xtr_path = "".join([base_path, "/data/Xtr.pickle"])

dispatch_id = ct.dispatch(training_workflow)(Xtr_path, 10, 10, 'sel', 2, 3, 'rx')

ct_results = ct.get_result(dispatch_id=dispatch_id, wait=True)
Xtr = ct_results.result

HTTPError: 500 Server Error: Internal Server Error for url: http://localhost:48008/api/submit

In [ ]:
training_workflow.cova_imports

In [20]:
Xtr_path = "".join([base_path, "/data/Xtr.pickle"])
x = training_workflow(Xtr_path, 10, 10, 'sel', 2, 3, 'rx')